In [22]:
import pandas as pd
from skimage.external import tifffile
import os
os.sys.path
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np

In [23]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'

foldersToLook = ['20190506-MSL decon rename', '20190607-MSL decon rename', '20190609-MSL decon rename', 
                 '20190625-MSL decon rename', '20190701-MSL decon rename', '20190712-MSL decon rename',
                 '20190716-MSL decon rename']

In [24]:
dataFrame = pd.DataFrame(columns = ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total','nucleusVolume', 'hetVolume'])

In [25]:
dataFrame

,File,Ch1_HetChrom,Ch1_Euchrom,Ch1_Total,Ch2_HetChrom,Ch2_Euchrom,Ch2_Total,Ch3_HetChrom,Ch3_Euchrom,Ch3_Total,nucleusVolume,hetVolume


In [26]:
def setinDF (folderName):
    currDir = f'{bigDir}\\{folderName}'
    currOrgCellFolder = f'{currDir}\\data'
    currCellFolder = f'{currDir}\\data_tiff'
    currNucFolder = f'{currDir}\\segmentation'
    currHetFolder = f'{currDir}\\segmentation_20_revised'
    
    nucFiles = []
    cellFiles = []
    hetFiles = []
    orgFiles = []

    for (dirpath, dirnames, filenames) in os.walk(currOrgCellFolder):
        orgFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currNucFolder):
        nucFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currCellFolder):
        cellFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currHetFolder):
        hetFiles.extend(filenames)
        break
        
    nucleusMasks = [f for f in nucFiles if "nucleus" in str(f)]
    
    if 'Thumbs.db' in hetFiles:
        hetFiles.remove('Thumbs.db')
    elif 'Thumbs.db' in cellFiles:
        cellFiles.remove('Thumbs.db')
    elif 'Thumbs.db' in orgFiles:
        orgFiles.remove('Thumbs.db')
    
    df = pd.DataFrame(columns = ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total', 'nucleusVolume', 'hetVolume'])
    
    for i, file in enumerate(orgFiles):
        df.loc[i, "File"] = file
    
    for i in range(len(orgFiles)):
        channel1 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 0)
        channel2 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 1)
        channel3 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 2)
        
        df.loc[i, 'Ch1_Total'] = channel1[0]
        df.loc[i, 'Ch1_HetChrom'] = channel1[1]
        df.loc[i, 'Ch1_Euchrom'] = channel1[2]
        
        df.loc[i, 'Ch2_Total'] = channel2[0]
        df.loc[i, 'Ch2_HetChrom'] = channel2[1]
        df.loc[i, 'Ch2_Euchrom'] = channel2[2]
        
        df.loc[i, 'Ch3_Total'] = channel3[0]
        df.loc[i, 'Ch3_HetChrom'] = channel3[1]
        df.loc[i, 'Ch3_Euchrom'] = channel3[2]
        
        df.loc[i, 'nucleusVolume'] = channel1[3]
        df.loc[i, 'hetVolume'] = channel1[4]
        
    
    return df

In [27]:
def extractData (currDir, cell, het, nuc, channel):
    myImage = tifffile.imread(f'{currDir}\\data_tiff\\{cell}')

    myChannel = myImage[:, channel, :, :]

    myNucleusMask = tifffile.imread(f'{currDir}\\segmentation\\{nuc}')
    myNucleusMask = myNucleusMask.astype(bool)

    myHeteroMask = tifffile.imread(f'{currDir}\\segmentation_20_revised\\{het}')
    myHeteroMask = myHeteroMask.astype(bool)
    
    myTotal = np.sum(myChannel[myNucleusMask])
    myHet = np.sum(myChannel[myHeteroMask])
    
    myTotalVolume = np.sum(myChannel[myNucleusMask])
    myHetVolume = np.sum(myChannel[myHeteroMask])
    
    myEuchro = myTotal - myHet

    result = []
    result.append(myTotal)
    result.append(myHet)
    result.append(myEuchro)
    result.append(myTotalVolume)
    result.append(myHetVolume)
    
    return result

In [28]:
for i in range(len(foldersToLook)):
    print(foldersToLook[i])
    df_set =  setinDF(foldersToLook[i])
    dataFrame = dataFrame.merge(df_set, how = 'outer', on = ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total', 'nucleusVolume', 'hetVolume'])

20190506-MSL decon rename
20190607-MSL decon rename
20190609-MSL decon rename
20190625-MSL decon rename
20190701-MSL decon rename
20190712-MSL decon rename
20190716-MSL decon rename


In [29]:
dataFrame

,File,Ch1_HetChrom,Ch1_Euchrom,Ch1_Total,Ch2_HetChrom,Ch2_Euchrom,Ch2_Total,Ch3_HetChrom,Ch3_Euchrom,Ch3_Total,nucleusVolume,hetVolume
0,20190506_cellde-01.czi.ims,1.969754e+07,6.088067e+07,8.057821e+07,3.859660e+07,6.529230e+07,1.038889e+08,6.677110e+07,2.039683e+08,2.707394e+08,8.057821e+07,1.969754e+07
1,20190506_cellde-02.czi.ims,1.518955e+07,4.444006e+07,5.962961e+07,2.507208e+07,4.077003e+07,6.584212e+07,6.935493e+07,2.105371e+08,2.798920e+08,5.962961e+07,1.518955e+07
2,20190506_cellde-03.czi.ims,2.104793e+07,6.406262e+07,8.511054e+07,4.637384e+07,7.868197e+07,1.250558e+08,7.536676e+07,2.345814e+08,3.099481e+08,8.511054e+07,2.104793e+07
3,20190506_cellde-04.czi.ims,1.570845e+07,4.784765e+07,6.355609e+07,3.147956e+07,5.416507e+07,8.564462e+07,6.010725e+07,1.930500e+08,2.531572e+08,6.355609e+07,1.570845e+07
4,20190506_cellde-05.czi.ims,1.454293e+07,4.500368e+07,5.954662e+07,3.405421e+07,5.889526e+07,9.294946e+07,5.782878e+07,1.895469e+08,2.473757e+08,5.954662e+07,1.454293e+07
5,20190506_cellde-06.czi.ims,3.993018e+07,1.350927e+08,1.750229e+08,2.814923e+07,4.104730e+07,6.919652e+07,1.205326e+08,4.558351e+08,5.763677e+08,1.750229e+08,3.993018e+07
6,20190506_cellde-07.czi.ims,3.389897e+07,1.220329e+08,1.559319e+08,2.472162e+07,3.599572e+07,6.071734e+07,1.027255e+08,3.985886e+08,5.013141e+08,1.559319e+08,3.389897e+07
7,20190506_cellde-08.czi.ims,2.768061e+07,7.789395e+07,1.055746e+08,8.045079e+07,1.419004e+08,2.223512e+08,6.110240e+07,1.874302e+08,2.485326e+08,1.055746e+08,2.768061e+07
8,20190506_cellde-09.ims,3.482062e+07,7.263430e+07,1.074549e+08,1.004354e+08,1.299763e+08,2.304118e+08,8.463761e+07,2.208202e+08,3.054578e+08,1.074549e+08,3.482062e+07
9,20190506_cellde-10.czi.ims,3.257213e+07,9.394998e+07,1.265221e+08,7.824398e+07,1.396978e+08,2.179417e+08,5.785909e+07,1.866084e+08,2.444675e+08,1.265221e+08,3.257213e+07


In [11]:
dataFrame.to_csv(r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\decon_data_perChannel_v2.csv')